# EDA - Optimización del Talento (ABC Corporation)

## Objetivo del EDA
- Entender el dataset: tamaño, tipos, nulos, duplicados, valores constantes y calidad general.
- Identificar variables potencialmente relacionadas con `Attrition` (rotación).
- Definir qué limpieza/transformaciones serán necesarias en la siguiente fase.

> Nota: En este notebook NO hacemos limpieza permanente. Solo analizamos y documentamos decisiones.

In [1]:
# Imports y configuración previa

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

## Carga del dataset
Cargamos el CSV desde `data/raw/`.

In [2]:
path = "../data/raw/hr.csv"
df = pd.read_csv(path)

df.shape, df.head()

((1474, 35),
     Age Attrition     BusinessTravel  DailyRate              Department  \
 0 41.00       Yes      Travel_Rarely       1102                   Sales   
 1 49.00        No  Travel_Frequently        279  Research & Development   
 2 37.00       Yes      Travel_Rarely       1373  Research & Development   
 3 33.00        No  Travel_Frequently       1392  Research & Development   
 4 27.00        No      Travel_Rarely        591  Research & Development   
 
    DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
 0                 1          2  Life Sciences              1               1   
 1                 8          1  Life Sciences              1               2   
 2                 2          2          Other              1               4   
 3                 3          4  Life Sciences              1               5   
 4                 2          1        Medical              1               7   
 
    EnvironmentSatisfaction  Gender  Hour

In [3]:
# Vista general del dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1401 non-null   float64
 1   Attrition                 1474 non-null   object 
 2   BusinessTravel            1357 non-null   object 
 3   DailyRate                 1474 non-null   int64  
 4   Department                1445 non-null   object 
 5   DistanceFromHome          1474 non-null   int64  
 6   Education                 1474 non-null   int64  
 7   EducationField            1416 non-null   object 
 8   EmployeeCount             1474 non-null   int64  
 9   EmployeeNumber            1474 non-null   int64  
 10  EnvironmentSatisfaction   1474 non-null   int64  
 11  Gender                    1474 non-null   object 
 12  HourlyRate                1474 non-null   int64  
 13  JobInvolvement            1474 non-null   int64  
 14  JobLevel

### Comentarios
- ¿Qué columnas parecen numéricas?
- ¿Qué columnas son categóricas?
- ¿Hay columnas con nulos según `info()`?

In [4]:
df.columns.tolist()

['Age',
 'Attrition',
 'BusinessTravel',
 'DailyRate',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeCount',
 'EmployeeNumber',
 'EnvironmentSatisfaction',
 'Gender',
 'HourlyRate',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'MonthlyRate',
 'NumCompaniesWorked',
 'Over18',
 'OverTime',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StandardHours',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

In [5]:
df.sample(5, random_state=42)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
852,29.00,No,Travel_Rarely,1401,Research & Development,6,1,Medical,1,1192,2,Female,54,3,1,lABORATORY tECHNICIAN,4.00,Married,"3,131.00",26342,1,Y,No,13,3,1,80.00,1,10,5.00,3,10,8,0,8.00
184,53.00,No,Travel_Rarely,1084,Research & Development,13,2,Medical,1,250,4,Female,57,4,2,mANUFACTURING dIRECTOR,1.00,Divorced,"4,450.00",26250,1,Y,No,11,3,3,NaN,2,5,3.00,3,4,2,1,3.00
1261,38.00,No,Travel_Rarely,833,Research & Development,18,3,Medical,1,1766,2,Male,60,1,2,hEALTHCARE rEPRESENTATIVE,4.00,Married,"5,811.00",24539,3,Y,Yes,16,3,3,80.00,1,15,2.00,3,1,0,1,0.00
67,45.00,No,Travel_Rarely,1339,Research & Development,7,3,Life Sciences,1,86,2,Male,59,3,3,rESEARCH sCIENTIST,1.00,Divorced,"9,724.00",18787,2,Y,No,17,3,3,NaN,1,25,2.00,3,1,0,0,0.00
220,36.00,No,Travel_Rarely,1396,Research & Development,5,2,Life Sciences,1,304,4,Male,62,3,2,lABORATORY tECHNICIAN,2.00,Single,"5,914.00",9945,8,Y,No,16,3,4,80.00,0,16,3.00,4,13,11,3,7.00


## Valores nulos
Cuantificamos nulos por columna y porcentaje.
> En esta fase NO imputamos (todavía), solo documentamos.

In [6]:
null_count = df.isna().sum().sort_values(ascending=False)
null_pct = (df.isna().mean() * 100).sort_values(ascending=False)

nulls = pd.DataFrame({"null_count": null_count, "null_pct": null_pct})
nulls[nulls["null_count"] > 0]

,null_count,null_pct
StandardHours,164,11.13
YearsWithCurrManager,148,10.04
MaritalStatus,132,8.96
BusinessTravel,117,7.94
TrainingTimesLastYear,88,5.97
Age,73,4.95
EducationField,58,3.93
OverTime,44,2.99
JobSatisfaction,29,1.97
Department,29,1.97


### Decisión (pendiente)
- Por ahora, los nulos se mantienen.
- Se registran columnas críticas para decidir imputación en la fase de limpieza.

## Duplicados
1) Filas duplicadas exactas
2) Duplicados de `EmployeeNumber` (id del empleado)

In [7]:
dup_rows = df.duplicated().sum()
dup_rows

np.int64(4)

In [8]:
df[df.duplicated()].head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1470,28.00,No,Travel_Rarely,866,Sales,5,3,Medical,1,1469,4,Male,84,3,2,sALES eXECUTIVE,1.00,Single,"8,463.00",23490,0,Y,No,18,3,4,NaN,0,6,4.00,3,5,4,1,NaN
1471,53.00,No,Travel_Rarely,1084,Research & Development,13,2,Medical,1,250,4,Female,57,4,2,mANUFACTURING dIRECTOR,1.00,Divorced,"4,450.00",26250,1,Y,No,11,3,3,NaN,2,5,3.00,3,4,2,1,3.00
1472,24.00,Yes,Travel_Rarely,240,Human Resources,22,1,Human Resources,1,1714,4,Male,58,1,1,hUMAN rESOURCES,3.00,Married,"1,555.00",11585,1,Y,No,11,3,3,80.00,1,1,2.00,3,1,0,0,0.00
1473,45.00,No,Travel_Rarely,1339,Research & Development,7,3,Life Sciences,1,86,2,Male,59,3,3,rESEARCH sCIENTIST,1.00,Divorced,"9,724.00",18787,2,Y,No,17,3,3,NaN,1,25,2.00,3,1,0,0,0.00


In [9]:
dup_emp = df.duplicated(subset=["EmployeeNumber"]).sum()
dup_emp

np.int64(4)

In [10]:
df[df.duplicated(subset=["EmployeeNumber"], keep=False)].sort_values("EmployeeNumber")

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
67,45.00,No,Travel_Rarely,1339,Research & Development,7,3,Life Sciences,1,86,2,Male,59,3,3,rESEARCH sCIENTIST,1.00,Divorced,"9,724.00",18787,2,Y,No,17,3,3,NaN,1,25,2.00,3,1,0,0,0.00
1473,45.00,No,Travel_Rarely,1339,Research & Development,7,3,Life Sciences,1,86,2,Male,59,3,3,rESEARCH sCIENTIST,1.00,Divorced,"9,724.00",18787,2,Y,No,17,3,3,NaN,1,25,2.00,3,1,0,0,0.00
1471,53.00,No,Travel_Rarely,1084,Research & Development,13,2,Medical,1,250,4,Female,57,4,2,mANUFACTURING dIRECTOR,1.00,Divorced,"4,450.00",26250,1,Y,No,11,3,3,NaN,2,5,3.00,3,4,2,1,3.00
184,53.00,No,Travel_Rarely,1084,Research & Development,13,2,Medical,1,250,4,Female,57,4,2,mANUFACTURING dIRECTOR,1.00,Divorced,"4,450.00",26250,1,Y,No,11,3,3,NaN,2,5,3.00,3,4,2,1,3.00
1470,28.00,No,Travel_Rarely,866,Sales,5,3,Medical,1,1469,4,Male,84,3,2,sALES eXECUTIVE,1.00,Single,"8,463.00",23490,0,Y,No,18,3,4,NaN,0,6,4.00,3,5,4,1,NaN
1041,28.00,No,Travel_Rarely,866,Sales,5,3,Medical,1,1469,4,Male,84,3,2,sALES eXECUTIVE,1.00,Single,"8,463.00",23490,0,Y,No,18,3,4,NaN,0,6,4.00,3,5,4,1,NaN
1222,24.00,Yes,Travel_Rarely,240,Human Resources,22,1,Human Resources,1,1714,4,Male,58,1,1,hUMAN rESOURCES,3.00,Married,"1,555.00",11585,1,Y,No,11,3,3,80.00,1,1,2.00,3,1,0,0,0.00
1472,24.00,Yes,Travel_Rarely,240,Human Resources,22,1,Human Resources,1,1714,4,Male,58,1,1,hUMAN rESOURCES,3.00,Married,"1,555.00",11585,1,Y,No,11,3,3,80.00,1,1,2.00,3,1,0,0,0.00


### Decisión propuesta
- Si se repite `EmployeeNumber` y las filas son idénticas: conservar una fila y eliminar duplicados en fase de limpieza.
- Motivo: `EmployeeNumber` debería ser único. Un índice duplicado complica el análisis y la inserción en BBDD.

## Columnas con un solo valor (constantes)
Detectamos columnas que no aportan información.

In [11]:
nunique = df.nunique(dropna=False).sort_values()
nunique.head(10)

EmployeeCount        1
Over18               1
Gender               2
Attrition            2
PerformanceRating    2
StandardHours        2
OverTime             3
BusinessTravel       4
JobInvolvement       4
WorkLifeBalance      4
dtype: int64

### Decisión
- Eliminar columnas constantes o sin variabilidad: `Over18`, `EmployeeCount`.
- `StandardHours`: solo tiene valor 80 cuando no es nulo -> se elimina también.

## Variable objetivo: Attrition
Analizamos proporción de empleados que se han ido vs los que se han quedado.

In [13]:
df["Attrition"].value_counts(dropna=False)

Attrition
No     1236
Yes     238
Name: count, dtype: int64

In [14]:
df["Attrition"].value_counts(normalize=True).mul(100).round(2)

Attrition
No    83.85
Yes   16.15
Name: proportion, dtype: float64

In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,"1,401.00",36.94,9.11,18.00,30.00,36.00,43.00,60.00
DailyRate,"1,474.00",802.70,403.54,102.00,465.00,803.00,"1,157.00","1,499.00"
DistanceFromHome,"1,474.00",9.20,8.10,1.00,2.00,7.00,14.00,29.00
Education,"1,474.00",2.91,1.02,1.00,2.00,3.00,4.00,5.00
EmployeeCount,"1,474.00",1.00,0.00,1.00,1.00,1.00,1.00,1.00
EmployeeNumber,"1,474.00","1,024.47",602.42,1.00,488.75,"1,020.50","1,555.75","2,068.00"
EnvironmentSatisfaction,"1,474.00",2.72,1.09,1.00,2.00,3.00,4.00,4.00
HourlyRate,"1,474.00",65.89,20.31,30.00,48.00,66.00,83.75,100.00
JobInvolvement,"1,474.00",2.73,0.71,1.00,2.00,3.00,3.00,4.00
JobLevel,"1,474.00",2.06,1.11,1.00,1.00,2.00,3.00,5.00


In [17]:
df.select_dtypes(include=np.number).skew().sort_values(ascending=False).head(10)

YearsSinceLastPromotion   1.99
PerformanceRating         1.93
YearsAtCompany            1.77
MonthlyIncome             1.37
TotalWorkingYears         1.12
NumCompaniesWorked        1.03
JobLevel                  1.03
StockOptionLevel          0.97
DistanceFromHome          0.96
YearsInCurrentRole        0.92
dtype: float64

### Comentarios
- ¿Hay valores extremos llamativos?
- ¿Alguna variable parece “rara” (por ejemplo, con rangos inesperados)?

In [18]:
cat_cols = df.select_dtypes(include="object").columns
cat_cols

Index(['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender',
       'JobRole', 'MaritalStatus', 'Over18', 'OverTime'],
      dtype='object')

In [19]:
for col in cat_cols:
    print(f"\n--- {col} ---")
    display(df[col].value_counts(dropna=False).head(15))


--- Attrition ---


Attrition
No     1236
Yes     238
Name: count, dtype: int64


--- BusinessTravel ---


BusinessTravel
Travel_Rarely        955
Travel_Frequently    263
Non-Travel           139
NaN                  117
Name: count, dtype: int64


--- Department ---


...

MaritalStatus
Married     604
Single      437
Divorced    298
NaN         132
Marreid       3
Name: count, dtype: int64


--- Over18 ---


Over18
Y    1474
Name: count, dtype: int64


--- OverTime ---


OverTime
No     1025
Yes     405
NaN      44
Name: count, dtype: int64

In [20]:
key_cat = ["OverTime", "BusinessTravel", "JobRole", "Department", "MaritalStatus", "Gender"]

for col in key_cat:
    if col in df.columns:
        ct = pd.crosstab(df[col], df["Attrition"], normalize="index").mul(100).round(1)
        print(f"\n% Attrition por {col}")
        display(ct)


% Attrition por OverTime


Attrition,No,Yes
OverTime,,
No,89.80,10.20
Yes,69.10,30.90



% Attrition por BusinessTravel


Attrition,No,Yes
BusinessTravel,,
Non-Travel,91.40,8.60
Travel_Frequently,75.30,24.70
Travel_Rarely,85.50,14.50



% Attrition por JobRole


...

Attrition,No,Yes
Department,,
Human Resources,79.70,20.30
Research & Development,86.60,13.40
Sales,79.80,20.20



% Attrition por MaritalStatus


Attrition,No,Yes
MaritalStatus,,
Divorced,89.30,10.70
Marreid,100.00,0.00
Married,87.10,12.90
Single,74.40,25.60



% Attrition por Gender


Attrition,No,Yes
Gender,,
Female,85.20,14.80
Male,82.90,17.10


### Interpretación
- ¿En qué categorías sube claramente el % de Attrition?
- Apunta 2 hipótesis (sin afirmar causalidad).

In [21]:
key_num = ["Age", "MonthlyIncome", "TotalWorkingYears", "DistanceFromHome", "YearsAtCompany"]

for col in key_num:
    if col in df.columns:
        display(df.groupby("Attrition")[col].describe().T)

Attrition,No,Yes
count,"1,180.00",221.00
mean,37.56,33.63
std,8.85,9.71
min,18.00,18.00
25%,31.00,28.00
50%,36.00,32.00
75%,43.00,39.00
max,60.00,58.00


Attrition,No,Yes
count,"1,226.00",234.00
mean,"6,832.69","4,741.03"
std,"4,813.70","3,628.36"
min,"1,051.00","1,009.00"
25%,"3,211.25","2,366.25"
50%,"5,205.00","3,187.00"
75%,"8,836.25","5,824.25"
max,"19,999.00","19,859.00"


Attrition,No,Yes
count,"1,236.00",238.00
mean,11.86,8.21
std,7.76,7.17
min,0.00,0.00
25%,6.00,3.00
50%,10.00,7.00
75%,16.00,10.00
max,38.00,40.00


Attrition,No,Yes
count,"1,236.00",238.00
mean,8.91,10.68
std,8.00,8.47
min,1.00,1.00
25%,2.00,3.00
50%,7.00,9.00
75%,13.00,17.00
max,29.00,29.00


Attrition,No,Yes
count,"1,236.00",238.00
mean,7.36,5.11
std,6.09,5.94
min,0.00,0.00
25%,3.00,1.00
50%,6.00,3.00
75%,10.00,7.00
max,37.00,40.00


# Conclusiones del EDA (para llevar a limpieza)
## Problemas detectados
- Nulos en columnas: (listar las principales)
- Duplicados (filas y EmployeeNumber)
- Columnas constantes (ruido)

## Decisiones para fase 2 (limpieza)
- Eliminar duplicados exactos y duplicados de EmployeeNumber conservando la primera fila
- Eliminar columnas constantes: Over18, EmployeeCount, StandardHours
- Nulos: pendiente de estrategia (clase siguiente)

## Hipótesis iniciales sobre Attrition (2-4 bullets)
- ...